# Script for crafting Adversarial Examples

In [9]:
import os

import utils.text_processing as tp

In [10]:
# get data from important word detector
%store -r important_words
%store -r sentence_packages

## ;Method, 3: ?punctuation

Plan:
im ersten Schritt: 
nach dem 
important Word , einfügen

#### 1. Create modified Dataset

In [3]:
original_sentences = []
for sentence in sentence_packages:
    original_sentences.append(sentence['original_sentence'])

In [4]:
modified_words = []
for word in important_words:
    modified_words.append(tp.to_punctuation(word))

In [5]:
modified_sentences = tp.generate_modified_sentences(original_sentences, important_words, modified_words)

#### 2. Import BERT Model

In [6]:
import torch
import random
import numpy as np
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model.eval();

#### 3. Prediction

In [7]:
original_predictions = []
for sentence in original_sentences:
    original_predictions.append(tp.predict_sentiment(model, tokenizer, sentence))

In [8]:
modified_predictions = []
for sentence in modified_sentences:
    modified_predictions.append(tp.predict_sentiment(model, tokenizer, sentence))

#### 4. Creation of Dataset

In [11]:
ds = pd.DataFrame(list(zip(original_sentences, original_predictions, modified_sentences, modified_predictions)),
                 columns = ['original_sentence', 'original_prediction', 'modified_sentence', 'modified_prediction'])

In [12]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   original_sentence    435 non-null    object
 1   original_prediction  435 non-null    int64 
 2   modified_sentence    435 non-null    object
 3   modified_prediction  435 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 13.7+ KB


#### 5. Creation of adversarial Dataset

In [13]:
advds = tp.filter_unchanged_predictions(ds)

In [19]:
advds.to_json(r'data/adversarial_dataset_punctuation.json', orient='records')

#### 6. Results

In [15]:
pmethod = 'punctuation'

In [16]:
results_punctuation = tp.generate_results_df(pmethod, ds, advds)
results_punctuation

,0
Perturbation Method,punctuation
Tokenizer,nlptown/bert-base-multilingual-uncased-sentiment
Model,nlptown/bert-base-multilingual-uncased-sentiment
Dataset,TripAdvisor Hotel Reviews
Output lables,Range from 0 to 4 - 0 = NEG; 4 = POS
Items in original dataset,435
Items in adversarial dataset,56
Percentage,12.8736


In [17]:
%store results_punctuation

Stored 'results_punctuation' (DataFrame)
